# Generate test prompts for the variant `LLMSeqPromptRank` (Section 4.4)

In [ ]:
import pandas as pd
import numpy as np
import json
import random
import tiktoken
from main.data.session_dataset import SessionDataset
from main.llm_based.similarity_model.llm_seq_sim import LLMSeqSim
import pickle

In [ ]:
DATASET: Literal["beauty", "steam"] = "beauty"

# Name of the pickle with the test data for Beauty.
TEST_DATA_PICKLE_NAME = f"{DATASET}_dataset.pickle"

OTHER_MODEL_RECOMMENDATIONS = f"llmseqsim_{DATASET}_recommendations.pkl"

In [ ]:
dataset: SessionDataset = SessionDataset.from_pickle(open(TEST_DATA_PICKLE_NAME, "rb"))

In [ ]:
config = {
    "max_session_length_for_decay_precomputation": 500,
    "filter_prompt_items": True,
    "batch_size": 500,
    "dim_reduction_config": {
        "reduction_config": {
            "config": {},
            "reduction_technique": "pca"
        },
    "normalize": True,
    "reduced_dim_size": 512
  },
  "is_verbose": True,
  "cores": 15,
  "similarity_measure": "cosine",
  "embedding_combination_strategy": "mean",
  "combination_decay": "harmonic"
}

In [ ]:
model: LLMSeqSim = LLMSeqSim(**config)

model.train(dataset.get_train_data(), dataset.get_item_data())

In [ ]:
TOP_K = 20

recommendations: dict[int, np.ndarray] = model.predict(
    dataset.get_test_prompts(), top_k=TOP_K
)

In [ ]:
pickle.dump(recommendations, open(OTHER_MODEL_RECOMMENDATIONS, "wb"))